In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd

In [3]:
dat = pd.read_csv("train.csv")

In [4]:
print(dat.columns)

Index(['row_id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')


In [5]:
from datetime import datetime

dates = dat.date
datelist = []
for i in dates:
    datelist.append(datetime.strptime(i, '%Y-%m-%d'))

In [6]:
weekdaylist = []
for i in datelist:
    weekdaylist.append(i.weekday())

In [7]:
yearportion = []
for i in datelist:
    yearportion.append(i.timetuple().tm_yday / 366)


In [8]:
# add weekdaylist and yearportion to dat
weekday = pd.DataFrame(weekdaylist, columns = ["weekday"])
yearportion = pd.DataFrame(yearportion, columns = ["yearportion"])
newdat = pd.concat([dat, weekday, yearportion], axis = 1)

In [14]:
newdat.drop(labels = ["date"], axis = 1)
newdat

,row_id,date,num_sold,yearportion,country_Belgium,country_France,country_Germany,country_Italy,country_Poland,country_Spain,...,product_Kaggle Getting Started,product_Kaggle Recipe Book,product_Kaggle for Kids: One Smart Goose,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,2017-01-01,663,0.002732,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,2017-01-01,615,0.002732,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,2,2017-01-01,480,0.002732,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,3,2017-01-01,710,0.002732,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,4,2017-01-01,240,0.002732,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70123,70123,2020-12-31,614,1.000000,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
70124,70124,2020-12-31,215,1.000000,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
70125,70125,2020-12-31,158,1.000000,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
70126,70126,2020-12-31,135,1.000000,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0


In [13]:
# split country, store, product, and weekday
newdat = pd.get_dummies(newdat, columns = ["country", "store", "product", "weekday"])

KeyError: "None of [Index(['country', 'store', 'product', 'weekday'], dtype='object')] are in the [columns]"

Clean up data

In [15]:
y = newdat.num_sold
x = newdat.drop(labels = ["date", "row_id", "num_sold"], axis = 1)
print(len(y))

70128


In [21]:
n = len(y)
ntrain = 60000

x_train = x.iloc[:ntrain]
x_test = x.iloc[ntrain:]
y_train = y.iloc[:ntrain]
y_test = y.iloc[ntrain:]

print(x_train.shape)

(60000, 20)


In [24]:
# modela has 1 layers of 100 nodes
modela = tf.keras.Sequential()
modela.add(tf.keras.layers.Dense(100, input_shape=(20,), activation = "ReLU"))
modela.add(tf.keras.layers.Dense(1))

modela.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MAPE,
              metrics=['mae'])

modela.fit(x_train, y_train, batch_size = 50, epochs = 10)

Epoch 1/10
1200/1200 [==============================] - 2s 969us/step - loss: 46.5508 - mae: 104.2956
Epoch 2/10
1200/1200 [==============================] - 1s 1ms/step - loss: 18.9747 - mae: 42.8003
Epoch 3/10
1200/1200 [==============================] - 1s 1ms/step - loss: 15.4603 - mae: 34.1385
Epoch 4/10
1200/1200 [==============================] - 1s 886us/step - loss: 13.6267 - mae: 28.9691
Epoch 5/10
1200/1200 [==============================] - 1s 955us/step - loss: 13.0137 - mae: 27.0645
Epoch 6/10
1200/1200 [==============================] - 1s 879us/step - loss: 12.7026 - mae: 26.2760
Epoch 7/10
1200/1200 [==============================] - 1s 928us/step - loss: 12.4924 - mae: 25.7867
Epoch 8/10
1200/1200 [==============================] - 1s 898us/step - loss: 12.3216 - mae: 25.4460
Epoch 9/10
1200/1200 [==============================] - 1s 821us/step - loss: 12.1715 - mae: 25.0874
Epoch 10/10
1200/1200 [==============================] - 1s 907us/step - loss: 11.9815 - mae: 

In [26]:
# modela has 2 layers of 100, 100, nodes, batch size of 50
modelb = tf.keras.Sequential()
modelb.add(tf.keras.layers.Dense(100, input_shape=(20,), activation = "ReLU"))
modelb.add(tf.keras.layers.Dense(100, activation = "ReLU"))
modelb.add(tf.keras.layers.Dense(1))

modelb.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MAPE,
              metrics=['mse'])

modelb.fit(x_train, y_train, batch_size = 50, epochs = 10)

Epoch 1/10
1200/1200 [==============================] - 2s 1ms/step - loss: 22.4282 - mse: 7984.5396
Epoch 2/10
1200/1200 [==============================] - 1s 1ms/step - loss: 11.3202 - mse: 1861.9252
Epoch 3/10
1200/1200 [==============================] - 1s 1ms/step - loss: 10.6383 - mse: 1711.4318
Epoch 4/10
1200/1200 [==============================] - 1s 1ms/step - loss: 10.2813 - mse: 1627.1471
Epoch 5/10
1200/1200 [==============================] - 1s 995us/step - loss: 9.9522 - mse: 1552.5829
Epoch 6/10
1200/1200 [==============================] - 1s 1ms/step - loss: 9.7350 - mse: 1501.8373
Epoch 7/10
1200/1200 [==============================] - 1s 1ms/step - loss: 9.6132 - mse: 1473.1932
Epoch 8/10
1200/1200 [==============================] - 1s 1ms/step - loss: 9.5278 - mse: 1452.0547
Epoch 9/10
1200/1200 [==============================] - 1s 924us/step - loss: 9.4791 - mse: 1438.9436
Epoch 10/10
1200/1200 [==============================] - 1s 944us/step - loss: 9.4283 - mse:

In [33]:
# modelc has more layers
modelc = tf.keras.Sequential()
modelc.add(tf.keras.layers.Dense(100, input_shape=(20,), activation = "ReLU"))
modelc.add(tf.keras.layers.Dense(200, activation = "ReLU"))
modelc.add(tf.keras.layers.Dense(50, activation = "ReLU"))
modelc.add(tf.keras.layers.Dense(20, activation = "ReLU"))
modelc.add(tf.keras.layers.Dense(1))

modelc.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MAPE,
              metrics=['mae'])

modelc.fit(x_train, y_train, batch_size = 50, epochs = 15)

Epoch 1/15
1200/1200 [==============================] - 2s 1ms/step - loss: 16.6785 - mae: 34.2135
Epoch 2/15
1200/1200 [==============================] - 1s 1ms/step - loss: 10.3693 - mae: 20.9571
Epoch 3/15
1200/1200 [==============================] - 1s 1ms/step - loss: 9.9769 - mae: 20.2209
Epoch 4/15
1200/1200 [==============================] - 1s 1ms/step - loss: 9.8081 - mae: 19.8968
Epoch 5/15
1200/1200 [==============================] - 1s 1ms/step - loss: 9.6746 - mae: 19.6137
Epoch 6/15
1200/1200 [==============================] - 1s 1ms/step - loss: 9.6286 - mae: 19.4961
Epoch 7/15
1200/1200 [==============================] - 1s 1ms/step - loss: 9.4576 - mae: 19.1882
Epoch 8/15
1200/1200 [==============================] - 1s 1ms/step - loss: 9.3902 - mae: 19.0291
Epoch 9/15
1200/1200 [==============================] - 2s 1ms/step - loss: 9.3765 - mae: 18.9637
Epoch 10/15
1200/1200 [==============================] - 2s 1ms/step - loss: 9.3160 - mae: 18.8797
Epoch 11/15
1200/

In [34]:
print(modela.evaluate(x_test, y_test)[0])
print(modelb.evaluate(x_test, y_test)[0])
print(modelc.evaluate(x_test, y_test)[0])

317/317 [==============================] - 0s 1ms/step - loss: 25.5377 - mae: 58.5604
25.53772735595703
317/317 [==============================] - 0s 881us/step - loss: 24.3277 - mse: 7317.1328
24.327674865722656
317/317 [==============================] - 0s 922us/step - loss: 24.4657 - mae: 55.7490
24.465726852416992


Import test files and clean up

In [30]:
testdat = pd.read_csv("test.csv")

# perform same actions as on training data
from datetime import datetime

dates = testdat.date
datelist = []
for i in dates:
    datelist.append(datetime.strptime(i, '%Y-%m-%d'))

weekdaylist = []
for i in datelist:
    weekdaylist.append(i.weekday())

yearportion = []
for i in datelist:
    yearportion.append(i.timetuple().tm_yday / 366)

# add weekdaylist and yearportion to dat
weekday = pd.DataFrame(weekdaylist, columns = ["weekday"])
yearportion = pd.DataFrame(yearportion, columns = ["yearportion"])
testdat = pd.concat([testdat, weekday, yearportion], axis = 1)
testdat = testdat.drop(labels = ["date"], axis = 1)


# split country, store, product, and weekday
testdat = pd.get_dummies(testdat, columns = ["country", "store", "product", "weekday"])

rowids = testdat.row_id

testdat = testdat.drop(labels = ["row_id"], axis = 1)
testdat

,yearportion,country_Belgium,country_France,country_Germany,country_Italy,country_Poland,country_Spain,store_KaggleMart,store_KaggleRama,product_Kaggle Advanced Techniques,product_Kaggle Getting Started,product_Kaggle Recipe Book,product_Kaggle for Kids: One Smart Goose,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0.002732,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
1,0.002732,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
2,0.002732,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
3,0.002732,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
4,0.002732,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,0.997268,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0
17516,0.997268,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0
17517,0.997268,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0
17518,0.997268,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0


In [35]:
predictions = modelc.predict(testdat)

In [36]:
row_ids = pd.DataFrame(rowids, columns = ["row_id"])
preds = pd.DataFrame(predictions, columns = ["num_sold"])

outdat = pd.concat([row_ids, preds], axis = 1)

outdat.to_csv("September_Kaggle_Submission3.csv", index = False)
